## Machine Learning Cycling Ride Classifier

## Outline

* [First Bullet Header](#first-bullet)
* [Second Bullet Header](#second-bullet)



In [122]:
import pandas as pd
import altair as alt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.io import arff

## Import Data

In [123]:
rides = pd.read_csv('/Users/chasekregor/GitHub/2020/CyclingRideClassifier/data/rides.csv')

## Quick Data EDA

In [124]:
rides.describe()

,activity_id,distance,elevation gain,average_speed,average_watts,suffer_score,average_heartrate,average_cadence,kilojoules,average_temp,start_longitude,start_latitude
count,7.240000e+02,724.000000,724.000000,724.000000,676.000000,616.000000,618.000000,535.000000,670.000000,487.000000,512.000000,512.000000
mean,1.469044e+09,15.397818,745.758925,222.438853,123.401331,67.900974,147.064563,78.487290,515.428507,21.080082,-99.553535,37.972109
std,1.121397e+09,12.320441,998.293526,3974.501586,36.227850,57.046422,16.528981,10.017512,410.220105,7.892524,18.218156,5.708634
min,1.042879e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-160.000000,-36.780000
25%,4.308549e+08,7.695000,0.000000,10.704859,106.075000,30.000000,140.650000,73.900000,268.525000,16.000000,-105.280000,36.100000
50%,1.024044e+09,13.000000,406.824147,14.070329,127.600000,53.000000,149.000000,79.800000,429.150000,22.000000,-105.010000,39.750000
75%,2.591429e+09,20.000000,1110.564304,17.050488,144.725000,87.000000,155.800000,85.500000,649.525000,27.000000,-86.850000,40.010000
max,3.736531e+09,101.890000,9558.070866,79200.071582,236.200000,383.000000,185.600000,101.000000,3193.000000,36.000000,175.000000,48.680000


In [125]:
rides[rides['average_speed']> 40]

,date,moving_time,activity_id,name,distance,elevation gain,type,trainer,average_speed,average_watts,suffer_score,average_heartrate,average_cadence,kilojoules,gear_id,average_temp,start_longitude,start_latitude,device_watts
386,2017-03-30,0 days 00:00:01.000000000,921397139,Spin with Abby and Mary Lou,20.0,0.0,Ride,False,72000.044739,NaN,NaN,NaN,NaN,NaN,b1315248,NaN,NaN,NaN,False
404,2016-12-31,0 days 00:00:01.000000000,816966884,Distance: Spin class with Abby's fam. Awesome ...,22.0,0.0,Ride,False,79200.071582,NaN,NaN,NaN,NaN,NaN,b1315248,NaN,NaN,NaN,False


## Data Preprocessing
Here the idea is to handle any missing or problematic data. In addition we are going to label and split our data into train, test, and validation splits.

# not sure why I have to run the cell below twice?
 

In [126]:
#getting rid of rides above 40, those are obviously data errors
rides = rides[rides['average_speed']< 40]


#rides['moving_time'] = rides['moving_time'].dt.total_seconds().div(60).astype(int)

#fill all the None's the average of their respective columns
rides = rides.fillna(rides.mean())

rides = rides.apply(lambda x: x.astype(str).str.lower())


#for some reason I can't get the none in average_temp to come out so I simply deleting those rows
rides = rides[rides['average_temp'] != 'none']

#changing some of the features we want to use to type float
rides.average_speed = rides.average_speed.astype(float)
rides.average_heartrate = rides.average_heartrate.astype(float)
rides.kilojoules = rides.kilojoules.astype(float)

rides = rides.replace('true',"1")
rides = rides.replace('false',"0")

rides.head()

,date,moving_time,activity_id,name,distance,elevation gain,type,trainer,average_speed,average_watts,suffer_score,average_heartrate,average_cadence,kilojoules,gear_id,average_temp,start_longitude,start_latitude,device_watts
0,2020-07-09,0 days 01:00:42.000000000,3736531260,group workout: ebbetts +2 w dad & hunter,18.24,0.0,ride,1,18.025233,154.6,72.0,151.2,87.9,563.0,b4933861,21.080082135523615,-99.55353515624923,37.972109375000045,1
1,2020-07-06,0 days 00:20:01.000000000,3724400147,ramp test,5.66,0.0,ride,1,16.953740,175.2,25.0,151.5,83.1,210.4,b4933861,21.080082135523615,-99.55353515624923,37.972109375000045,1
2,2020-06-21,0 days 00:55:34.000000000,3650328303,morning ride,13.17,219.81627296587925,ride,0,14.217967,92.3,19.0,126.8,76.9,307.8,b4933861,26.0,-104.97,39.74,1
3,2020-06-16,0 days 00:20:31.000000000,3626733948,pettit,5.97,0.0,ride,1,17.452577,153.7,12.0,137.2,85.7,189.3,b4933861,21.080082135523615,-99.55353515624923,37.972109375000045,1
4,2020-06-13,0 days 01:30:00.000000000,3609777201,junction -1: every now and then i think to mys...,25.48,0.0,ride,1,16.987294,161.2,157.0,159.5,88.8,870.7,b4933861,21.080082135523615,-99.55353515624923,37.972109375000045,1


## Labeling Data

In [127]:
conditions = [
    (rides['gear_id'] == 'b3092328'),
    (rides['gear_id'] == 'b5499491'),
    (rides['gear_id'] != 'b3092328') & (rides['gear_id'] != 'b5499491')
    ]
choices = ['mtb','peloton','road']
rides['classification'] = np.select(conditions, choices, default=None)
rides.classification.value_counts()

road       577
peloton     99
mtb         46
Name: classification, dtype: int64

## Splitting the Data

In [105]:
xVar = list(['distance','average_speed','average_heartrate','kilojoules','elevation gain','suffer_score','trainer','average_cadence','average_watts','average_temp','device_watts'])
yVar = rides['classification']
df2 = rides[xVar]

In [106]:
X_train, X_test, y_train, y_test = train_test_split(df2, yVar, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(577, 11) (577,)
(145, 11) (145,)


## ML Model

In [107]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)

clf.fit(X_train, y_train)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=2, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

y_pred = clf.predict(X_test)

print(pd.crosstab(y_test, y_pred, rownames=['Actual Result'], colnames=['Predicted Result']))
print("")

from sklearn.metrics import classification_report, accuracy_score

print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

print("")
list(zip(X_train, clf.feature_importances_))

Predicted Result  mtb  peloton  road
Actual Result                       
mtb                 6        0     1
peloton             0       11     4
road                0        5   118

              precision    recall  f1-score   support

         mtb       1.00      0.86      0.92         7
     peloton       0.69      0.73      0.71        15
        road       0.96      0.96      0.96       123

    accuracy                           0.93       145
   macro avg       0.88      0.85      0.86       145
weighted avg       0.93      0.93      0.93       145

0.9310344827586207



[('distance', 0.05752457093751591),
 ('average_speed', 0.2065260767866693),
 ('average_heartrate', 0.06309483119043494),
 ('kilojoules', 0.11320414847281976),
 ('elevation gain', 0.061339314697498794),
 ('suffer_score', 0.06231705958893115),
 ('trainer', 0.14834764738583936),
 ('average_cadence', 0.08088901452684039),
 ('average_watts', 0.09395007486922188),
 ('average_temp', 0.07790792251800607),
 ('device_watts', 0.034899339026222456)]

## Building an Ensemble Model

### k-Nearest Neighbors (k-NN)

In [108]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
#create new a knn model
knn = KNeighborsClassifier()
#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}
#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=5)
#fit model to training data
knn_gs.fit(X_train, y_train)
#save best model
knn_best = knn_gs.best_estimator_
#check best n_neigbors value
print(knn_gs.best_params_)

{'n_neighbors': 5}


### Random Forest

In [67]:
from sklearn.ensemble import RandomForestClassifier
#create a new random forest classifier
rf = RandomForestClassifier()
#create a dictionary of all values we want to test for n_estimators
params_rf = {'n_estimators': [50, 100, 200]}
#use gridsearch to test all values for n_estimators
rf_gs = GridSearchCV(rf, params_rf, cv=5)
#fit model to training data
rf_gs.fit(X_train, y_train)
#save best model
rf_best = rf_gs.best_estimator_
#check best n_estimators value
print(rf_gs.best_params_)

{'n_estimators': 100}


### Logistic Regression

In [68]:
from sklearn.linear_model import LogisticRegression
#create a new logistic regression model
log_reg = LogisticRegression()
#fit the model to the training data
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Support Vector Classifier

In [69]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

SVC = make_pipeline(StandardScaler(), SVC(gamma='auto'))

SVC.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

### Compare the models scores


In [70]:
#test the three models with the test data and print their accuracy scores
print('knn: {}'.format(knn_best.score(X_test, y_test)))
print('rf: {}'.format(rf_best.score(X_test, y_test)))
print('log_reg: {}'.format(log_reg.score(X_test, y_test)))
print('SVC: {}'.format(SVC.score(X_test, y_test)))

knn: 0.8068965517241379
rf: 0.896551724137931
log_reg: 0.8
SVC: 0.8620689655172413


## Voting Classifier

In [71]:
from sklearn.ensemble import VotingClassifier
#create a dictionary of our models
estimators=[('knn', knn_best), ('random forrests', rf_best), ('log_reg', log_reg), ('SVC', SVC)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

In [72]:
#fit model to training data
ensemble.fit(X_train, y_train)
#test our model on the test data
ensemble.score(X_test, y_test)

0.8620689655172413

In [74]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100, 10000, 3), max_iter=1000)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[  0   0  10]
 [  0   0  19]
 [  0   0 116]]
              precision    recall  f1-score   support

         mtb       0.00      0.00      0.00        10
     peloton       0.00      0.00      0.00        19
        road       0.80      1.00      0.89       116

    accuracy                           0.80       145
   macro avg       0.27      0.33      0.30       145
weighted avg       0.64      0.80      0.71       145

0.8


## Building a Neural Network with Keras and Tensorflow

In [131]:
xVar = list(['distance','average_speed','average_heartrate','kilojoules','elevation gain','suffer_score','trainer','average_cadence','average_watts','average_temp','device_watts','classification'])
rides = rides[xVar]
rides = rides.apply(pd.to_numeric, errors='coerce')
df["A"] = pd.to_numeric(df["A"], downcast="float")
rides

,distance,average_speed,average_heartrate,kilojoules,elevation gain,suffer_score,trainer,average_cadence,average_watts,average_temp,device_watts,classification
0,18.24,18.025233,151.200000,563.000000,0.000000,72.000000,1,87.90000,154.600000,21.080082,1,NaN
1,5.66,16.953740,151.500000,210.400000,0.000000,25.000000,1,83.10000,175.200000,21.080082,1,NaN
2,13.17,14.217967,126.800000,307.800000,219.816273,19.000000,0,76.90000,92.300000,26.000000,1,NaN
3,5.97,17.452577,137.200000,189.300000,0.000000,12.000000,1,85.70000,153.700000,21.080082,1,NaN
4,25.48,16.987294,159.500000,870.700000,0.000000,157.000000,1,88.80000,161.200000,21.080082,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
719,11.00,9.498031,147.064563,247.200000,298.228346,67.900974,0,78.48729,59.300000,21.080082,0,NaN
720,45.55,12.495526,147.064563,1341.600000,2792.979003,67.900974,0,78.48729,102.200000,21.080082,0,NaN
721,28.20,11.983268,147.064563,998.000000,2025.918635,67.900974,0,78.48729,117.800000,21.080082,0,NaN
722,20.24,9.634485,147.064563,1056.700000,2527.887139,67.900974,0,78.48729,139.700000,21.080082,0,NaN


In [132]:
rides.dtypes

distance             float64
average_speed        float64
average_heartrate    float64
kilojoules           float64
elevation gain       float64
suffer_score         float64
trainer                int64
average_cadence      float64
average_watts        float64
average_temp         float64
device_watts           int64
classification       float64
dtype: object

In [117]:
classification = rides.pop('classification')

In [118]:
dataset = tf.data.Dataset.from_tensor_slices((rides.values, classification.values))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [109]:
import tensorflow as tf

In [110]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='sigmoid', 
                          name='fc1', input_shape=(4,)),
    tf.keras.layers.Dense(3, name='fc2', activation='softmax')])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 16)                80        
_________________________________________________________________
fc2 (Dense)                  (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [111]:
model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

In [114]:
num_epochs = 100
training_size = 100
batch_size = 2
steps_per_epoch = np.ceil(training_size / batch_size)

ds_train = X_train.shuffle(buffer_size=training_size)
ds_train = ds_train.repeat()
ds_train = ds_train.batch(batch_size=batch_size)
ds_train = ds_train.prefetch(buffer_size=1000)

history = model.fit(ds_train, epochs=num_epochs,
                         steps_per_epoch=steps_per_epoch, 
                         verbose=0)

AttributeError: 'DataFrame' object has no attribute 'shuffle'

In [ ]:
hist = history.history

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(1, 2, 1)
ax.plot(hist['loss'], lw=3)
ax.set_title('Training loss', size=15)
ax.set_xlabel('Epoch', size=15)
ax.tick_params(axis='both', which='major', labelsize=15)

ax = fig.add_subplot(1, 2, 2)
ax.plot(hist['accuracy'], lw=3)
ax.set_title('Training accuracy', size=15)
ax.set_xlabel('Epoch', size=15)
ax.tick_params(axis='both', which='major', labelsize=15)
plt.tight_layout()
#plt.savefig('ch13-cls-learning-curve.pdf')

plt.show()